In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np
!pip install folium
import folium 

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.

Libraries imported.


In [3]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


In [8]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']

### Geographic data

In [10]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [11]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [12]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


##### boroughs and neighborhoods in NYC

In [13]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [14]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


##### Geo Map

In [24]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

##### Geo Map  (Manhattan)

In [16]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [18]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [19]:
CLIENT_ID = 'B05VP30MYOLDS2NLU3SHYMJMOIBJS3SHUYKMN3K3NOTM4RBK' # your Foursquare ID
CLIENT_SECRET = '3IOSKLHZZNJIDZJLHRVTC4PTNXDCHZPO2XEO4NWMDJ2SGAFV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: B05VP30MYOLDS2NLU3SHYMJMOIBJS3SHUYKMN3K3NOTM4RBK
CLIENT_SECRET:3IOSKLHZZNJIDZJLHRVTC4PTNXDCHZPO2XEO4NWMDJ2SGAFV


In [20]:
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

In [21]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


##### Foursquare

In [22]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=B05VP30MYOLDS2NLU3SHYMJMOIBJS3SHUYKMN3K3NOTM4RBK&client_secret=3IOSKLHZZNJIDZJLHRVTC4PTNXDCHZPO2XEO4NWMDJ2SGAFV&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

In [23]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '610f6b5fcaaf62352342d41e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 24,
  'suggestedBounds': {'ne': {'lat': 40.88105078329964,
    'lng': -73.90471933917806},
   'sw': {'lat': 40.87205077429964, 'lng': -73.91659997808156}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4baf59e8f964a520a6f93be3',
       'name': 'Bikram Yoga',
       'location': {'address': '5500 Broadway',
        'crossStreet': '230th Street',
        'lat': 40.876843690797934,
        'lng': -73.90620384419528,
        'labeledLatLngs': [{'la

In [25]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "Restaurant",
    "near": "Da Nang",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [26]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [27]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/srv/conda/envs/notebook/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Bikram Yoga,Yoga Studio,40.876844,-73.906204
1,Arturo's,Pizza Place,40.874412,-73.910271
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Dunkin',Donut Shop,40.877136,-73.906666
4,Starbucks,Coffee Shop,40.877531,-73.905582


In [28]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

24 venues were returned by Foursquare.


### Population Data

In [41]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# conda install -c anaconda beautiful-soup --yes
#from bs4 import BeautifulSoup # package for parsing HTML and XML documents
#pip install beautifulsoup4
!pip install bs4
from bs4 import BeautifulSoup

import csv # implements classes to read and write tabular data in CSV form

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.

Libraries imported.


In [47]:
!pip3 install lxml

     |████████████████████████████████| 6.3 MB 4.1 MB/s eta 0:00:01     |██                              | 409 kB 4.1 MB/s eta 0:00:02     |█████▋                          | 1.1 MB 4.1 MB/s eta 0:00:02     |██████████████████▎             | 3.6 MB 4.1 MB/s eta 0:00:01     |████████████████████████▍       | 4.8 MB 4.1 MB/s eta 0:00:01     |███████████████████████████▉    | 5.5 MB 4.1 MB/s eta 0:00:01


In [115]:
website_url = requests.get('https://en.wikipedia.org/wiki/Demographics_of_New_York_City').text
soup = BeautifulSoup(website_url,'html.parser')
table = soup.find('table',{'class':'wikitable sortable'})

headers = [header.text for header in table.find_all('th')]

table_rows = table.find_all('tr')        
rows = []
for row in table_rows:
   td = row.find_all('td')
   row = [row.text for row in td]
   rows.append(row)

with open('population.csv', 'w') as f:
   writer = csv.writer(f)
   writer.writerow(headers)
   writer.writerows(row for row in rows if row)

Population_data=pd.read_csv('population.csv')
Population_data.drop(Population_data.columns[[7,8,9,10,11]], axis=1,inplace=True)
print('Data downloaded!')

Data downloaded!


In [116]:
Population_data.columns = Population_data.columns.str.replace(' ', '')
Population_data.columns = Population_data.columns.str.replace('\'','')
#Population_data.rename(columns={'Borough':'persons_sq_mi','County':'persons_sq_km'}, inplace=True)
Population_data

,NewYorkCitysfiveboroughsvte\n,Jurisdiction\n,Population\n,GDP\n,Landarea\n,Density\n,Borough,persons/mi2,persons/km2\n
0,The Bronx\n,\n Bronx\n,"1,418,207\n",42.695\n,42.10\n,109.04\n,"33,867\n",NaN,NaN
1,Brooklyn\n,\n Kings\n,"2,559,903\n",91.559\n,70.82\n,183.42\n,"36,147\n",NaN,NaN
2,Manhattan\n,\n New York\n,"1,628,706\n",600.244\n,22.83\n,59.13\n,"71,341\n",NaN,NaN
3,Queens\n,\n Queens\n,"2,253,858\n",93.310\n,108.53\n,281.09\n,"20,767\n",NaN,NaN
4,Staten Island\n,\n Richmond\n,"476,143\n",14.514\n,58.37\n,151.18\n,"8,157\n",NaN,NaN
5,City of New York,"8,336,817",842.343,302.64,783.83,"27,547","10,636\n",NaN,NaN
6,State of New York,"19,453,561","1,731.910","47,126.40","122,056.82",412,159\n,NaN,NaN
7,Sources:[12][13][14] and see individual boroug...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
Population_data.rename(columns = {'NewYorkCitysfiveboroughsvte\n' : 'State',
                   'Jurisdiction\n':'County',
                   'Population\n':'Population', 
                   'GDP\n':'GDP',
                   'Landarea\n':'Landarea',
                    'Density\n':'Density'}, inplace=True)
Population_data

,State,County,Population,GDP,Landarea,Density,Borough,persons/mi2,persons/km2\n
0,The Bronx\n,\n Bronx\n,"1,418,207\n",42.695\n,42.10\n,109.04\n,"33,867\n",NaN,NaN
1,Brooklyn\n,\n Kings\n,"2,559,903\n",91.559\n,70.82\n,183.42\n,"36,147\n",NaN,NaN
2,Manhattan\n,\n New York\n,"1,628,706\n",600.244\n,22.83\n,59.13\n,"71,341\n",NaN,NaN
3,Queens\n,\n Queens\n,"2,253,858\n",93.310\n,108.53\n,281.09\n,"20,767\n",NaN,NaN
4,Staten Island\n,\n Richmond\n,"476,143\n",14.514\n,58.37\n,151.18\n,"8,157\n",NaN,NaN
5,City of New York,"8,336,817",842.343,302.64,783.83,"27,547","10,636\n",NaN,NaN
6,State of New York,"19,453,561","1,731.910","47,126.40","122,056.82",412,159\n,NaN,NaN
7,Sources:[12][13][14] and see individual boroug...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
Population_data['State']=Population_data['State'].replace(to_replace='\n', value='', regex=True)
Population_data['County']=Population_data['County'].replace(to_replace='\n', value='', regex=True)
Population_data['Population']=Population_data['Population'].replace(to_replace='\n', value='', regex=True)
Population_data['GDP']=Population_data['GDP'].replace(to_replace='\n', value='', regex=True)
Population_data['Landarea']=Population_data['Landarea'].replace(to_replace='\n', value='', regex=True)
Population_data['Density']=Population_data['Density'].replace(to_replace='\n', value='', regex=True)
Population_data['Borough']=Population_data['Borough'].replace(to_replace='\n', value='', regex=True)
Population_data

,State,County,Population,GDP,Landarea,Density,Borough,persons/mi2,persons/km2\n
0,The Bronx,Bronx,"1,418,207",42.695,42.10,109.04,"33,867",NaN,NaN
1,Brooklyn,Kings,"2,559,903",91.559,70.82,183.42,"36,147",NaN,NaN
2,Manhattan,New York,"1,628,706",600.244,22.83,59.13,"71,341",NaN,NaN
3,Queens,Queens,"2,253,858",93.310,108.53,281.09,"20,767",NaN,NaN
4,Staten Island,Richmond,"476,143",14.514,58.37,151.18,"8,157",NaN,NaN
5,City of New York,"8,336,817",842.343,302.64,783.83,"27,547","10,636",NaN,NaN
6,State of New York,"19,453,561","1,731.910","47,126.40","122,056.82",412,159,NaN,NaN
7,Sources:[12][13][14] and see individual boroug...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [119]:
Population_data = Population_data.fillna('')
Population_data

,State,County,Population,GDP,Landarea,Density,Borough,persons/mi2,persons/km2\n
0,The Bronx,Bronx,"1,418,207",42.695,42.10,109.04,"33,867",,
1,Brooklyn,Kings,"2,559,903",91.559,70.82,183.42,"36,147",,
2,Manhattan,New York,"1,628,706",600.244,22.83,59.13,"71,341",,
3,Queens,Queens,"2,253,858",93.310,108.53,281.09,"20,767",,
4,Staten Island,Richmond,"476,143",14.514,58.37,151.18,"8,157",,
5,City of New York,"8,336,817",842.343,302.64,783.83,"27,547","10,636",,
6,State of New York,"19,453,561","1,731.910","47,126.40","122,056.82",412,159,,
7,Sources:[12][13][14] and see individual boroug...,,,,,,,,


In [120]:
Population_data.drop(7)

,State,County,Population,GDP,Landarea,Density,Borough,persons/mi2,persons/km2\n
0,The Bronx,Bronx,"1,418,207",42.695,42.10,109.04,"33,867",,
1,Brooklyn,Kings,"2,559,903",91.559,70.82,183.42,"36,147",,
2,Manhattan,New York,"1,628,706",600.244,22.83,59.13,"71,341",,
3,Queens,Queens,"2,253,858",93.310,108.53,281.09,"20,767",,
4,Staten Island,Richmond,"476,143",14.514,58.37,151.18,"8,157",,
5,City of New York,"8,336,817",842.343,302.64,783.83,"27,547","10,636",,
6,State of New York,"19,453,561","1,731.910","47,126.40","122,056.82",412,159,,


### Cuisine Data

In [124]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import matplotlib.ticker as ticker

# notice: installing seaborn might takes a few minutes
!conda install -c anaconda seaborn -y
import seaborn as sns

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - seaborn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.10.14 |                0         128 KB  anaconda
    certifi-2020.6.20          |           py36_0         160 KB  anaconda
    seaborn-0.11.0             |             py_0         216 KB  anaconda
    ------------------------------------------------------------
                

In [125]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="Jupyter")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [127]:
# create map of New York City using latitude and longitude values
map_nyc = folium.Map(location=[latitude, longitude], zoom_start=10)

In [128]:
map_nyc